In [1]:
import os, pandas as pd, json, sys
import time
import openai
from tqdm import tqdm

In [2]:
df = pd.read_csv('./OpenAI_Json_query_output.csv')

In [3]:
def extract_content(output):
    # Try to load the string as JSON
    try:
        output_dict = json.loads(output)
        # Try to extract the "content" field
        if 'choices' in output_dict and output_dict['choices']:
            return output_dict['choices'][0]['message']['content']
    except (json.JSONDecodeError, TypeError, KeyError):
        # If it's not a JSON string or if the extraction fails, return the original string
        return output

In [4]:
df['output'] = df['output'].apply(extract_content)
df['output'] = df.apply(lambda row: row['output'] if pd.isnull(row['output']) or row['output'] == "" else row['output'], axis=1)

In [5]:
class TokenManager:
    def __init__(self, maximum_tokens_per_minute=10000):
        self.maximum_tokens_per_minute = maximum_tokens_per_minute
        self.tokens_utilized_in_the_last_minute = 0
        self.timestamp_of_the_last_request = time.time()
        self.total_tokens_used = 0  # Initialize total tokens used to 0

    def _reset_tokens_if_needed(self):
        current_timestamp = time.time()
        time_elapsed_since_last_request = current_timestamp - self.timestamp_of_the_last_request
        
        if time_elapsed_since_last_request >= 60:
            self.tokens_utilized_in_the_last_minute = 0
            self.timestamp_of_the_last_request = current_timestamp

    def calculate_tokens_available(self):
        self._reset_tokens_if_needed()
        tokens_available = self.maximum_tokens_per_minute - self.tokens_utilized_in_the_last_minute
        return tokens_available

    def register_tokens_used(self, tokens_used):
        if not isinstance(tokens_used, int) or tokens_used < 0:
            raise ValueError("Invalid token count")
        
        self._reset_tokens_if_needed()
        self.tokens_utilized_in_the_last_minute += tokens_used  # Update the tokens utilized in the last minute
        self.total_tokens_used += tokens_used  # Update the total tokens used
        
class OpenAIInteraction:
    def __init__(self, token_manager, output_csv=None, target_col=None, output_col_name=None, system_prompt=None, model="gpt-4"):
        self.token_manager = token_manager
        self.model = model
        self.output_csv = output_csv
        self.target_col = target_col
        self.output_col_name = output_col_name
        self.system_prompt = system_prompt

    def request_response(self, text_input):
        """Requests a response from the OpenAI model."""
        while self.token_manager.calculate_tokens_available() < 1200:
            time.sleep(1)
            print(f'Waiting for tokens to be available... ({self.token_manager.calculate_tokens_available()} tokens available)')

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": text_input}
        ]

        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=messages
            )
            tokens_used = response['usage']['total_tokens']
            self.token_manager.register_tokens_used(tokens_used)
            print(f'Response received, {tokens_used} tokens used, {self.token_manager.total_tokens_used} tokens used in total.')
            return response
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None

    def row_already_processed(self, row_value):
        """Checks if a row has already been processed based on the target column value."""
        if os.path.exists(self.output_csv):
            output_df = pd.read_csv(self.output_csv)
            return row_value in output_df[self.target_col].values
        return False

    def process_dataframe(self, dataframe):
        """Processes a DataFrame and outputs results to a CSV file."""
        if self.target_col not in dataframe.columns:
            print(f"Error: {self.target_col} does not exist in the dataframe")
            return

        if not os.path.exists(self.output_csv):
            with open(self.output_csv, 'w') as f:
                f.write(','.join(dataframe.columns.tolist() + [self.output_col_name]) + '\n')

        total_rows = len(dataframe)
        print(f'Processing {total_rows} rows...')

        with tqdm(total=total_rows * 2) as pbar:  # Multiply by 2 for two steps per row
            for _, row in dataframe.iterrows():
                text_input = row[self.target_col]
                if self.row_already_processed(text_input):
                    print(f'Skipping row {_}, already processed.')
                    pbar.update(2)  # Skip both steps for this row
                    continue

                response = self.request_response(text_input)
                pbar.update(1)  # Update progress bar after API call

                if response:
                    row_output = pd.DataFrame([row.tolist() + [response]], columns=dataframe.columns.tolist() + [self.output_col_name])
                    row_output.to_csv(self.output_csv, mode='a', header=False, index=False)
                    pbar.update(1)  # Update progress bar after writing to CSV
                else:
                    print(f'Failed to process row {_}.')
                    pbar.update(1)  # Update progress bar, even though writing to CSV failed
create_alternatives_prompt = """
Help create training materials for large text models by following these steps. Reply in a Natural Language Query style.

Steps:
1. Rewrite user input using different terms, keeping the original intent and mentioned metrics.
2. Don't reference JSON format or structure.
3. Ensure responses are only in Natural Language Query format.
4. Include minor typos in text, keeping ID's correct.
5. Retain all metrics from the original input.
6. Create two rephrased versions, labeled "Alternative 1" and "Alternative 2".
7. Include only these alternatives in your response, no other text.

Note:
- "Natural Language Query format" means rewording the query conversationally while keeping original intent and metrics.
- Minor typos can be misspellings or grammar errors, but should not cause confusion.
"""
# token_manager = TokenManager()
# interaction = OpenAIInteraction(token_manager, output_csv="./report_query_variations.csv", target_col="output", output_col_name="alternatives", system_prompt=create_alternatives_prompt)

In [6]:
# Commented out - this will process the entire dataframe, the previous run was only for 250 rows (3 hours runtime) - possible to consider running the remaining rows
# interaction.process_dataframe(df.head(250))

### Lets convert the generated alternatives, along with all orgininals into a single dataset

In [7]:
alternatives_df = pd.read_csv('./report_query_variations.csv')

In [8]:
def extract_content(json_str):
    try:
        # Parse the JSON string into a dictionary
        json_dict = json.loads(json_str)
        # Navigate to the "content" field and return its value
        content = json_dict['choices'][0]['message']['content']
        return content
    except (KeyError, IndexError, TypeError, json.JSONDecodeError):
        # Handle any errors that occur while parsing the JSON or navigating to the "content" field
        return None

# Use the apply method to apply the function to each row in the 'alternatives' column
alternatives_df['alternatives'] = alternatives_df['alternatives'].apply(extract_content)

In [9]:
# Merge the two DataFrames on common columns using an outer join
merged_df = pd.merge(df, alternatives_df, on=['name', 'code', 'output'], how='outer', suffixes=('', '_alt'))

In [10]:
def extract_content(json_str):
    try:
        # Assuming the JSON object is already a dictionary, no need to parse with json.loads
        json_dict = json_str
        # Navigate to the "content" field and return its value
        content = json_dict['choices'][0]['message']['content']
        return content
    except (KeyError, IndexError, TypeError):
        # Handle any errors that occur while navigating to the "content" field
        return None

# Use the apply method to apply the function to each row in the 'alternatives' column
merged_df['content'] = merged_df['alternatives'].apply(extract_content)


In [11]:
def extract_alternatives(alternatives_text):
    if not isinstance(alternatives_text, str):
        return None, None  # Return None for both alternatives if alternatives_text is not a string
    
    try:
        # Split the alternatives_text by newline character to separate the alternatives
        alternatives = alternatives_text.split('\n\n')
        
        # Extract the text for Alternative 1 and Alternative 2
        alternative_1 = alternatives[0].replace('Alternative 1: ', '')
        alternative_2 = alternatives[1].replace('Alternative 2: ', '')
        
        return alternative_1, alternative_2
    except IndexError:
        # Handle any errors that occur if the expected format is not found
        return None, None

# Use the apply method to apply the function to each row in the 'alternatives' column,
# and assign the results to the new columns 'alternative_1' and 'alternative_2'
merged_df['alternative_1'], merged_df['alternative_2'] = zip(*merged_df['alternatives'].apply(extract_alternatives))


In [12]:
merged_df.drop(columns=['content', 'alternatives'], inplace=True)
merged_df.columns.values

array(['name', 'code', 'output', 'alternative_1', 'alternative_2'],
      dtype=object)

In [13]:
# Replace the phrases "Alternative 1:" and "Alternative 2:" with an empty string in the respective columns
merged_df.loc[:, 'alternative_1'] = merged_df['alternative_1'].str.replace('Alternative 1:', '', regex=False)
merged_df.loc[:, 'alternative_2'] = merged_df['alternative_2'].str.replace('Alternative 2:', '', regex=False)

In [14]:
task_instruction = "Given the following Avantlink User Query (@AvantQueryEDW2), translate the query into a JSON query that can be used to query the Avantlink API."

### Report Query NLP Training Data Creation

In [15]:
# List to hold the dictionaries
report_query_data_list = []

# Prefix to append to each input field
prefix = '@AvantQueryEDW2 - '

# Loop through each row of the DataFrame
for index, row in merged_df.iterrows():
    # Create a dictionary for the 'output' column
    output_dict = {
        "instruction": task_instruction,
        "input": prefix + row['output'],
        "output": row['code']
    }
    report_query_data_list.append(output_dict)
    
    # If 'alternative_1' is not None, create a dictionary for the 'alternative_1' column
    if pd.notnull(row['alternative_1']):
        alt1_dict = {
            "instruction": task_instruction,
            "input": prefix + row['alternative_1'],
            "output": row['code']
        }
        report_query_data_list.append(alt1_dict)
    
    # If 'alternative_2' is not None, create a dictionary for the 'alternative_2' column
    if pd.notnull(row['alternative_2']):
        alt2_dict = {
            "instruction": task_instruction,
            "input": prefix + row['alternative_2'],
            "output": row['code']
        }
        report_query_data_list.append(alt2_dict)

In [16]:
# pd.DataFrame(data_list).to_csv('./data/training/training_data_edw2_queries.csv', index=False)

In [17]:
# report_query_data_list

In [18]:
prepared_columns = pd.read_csv('./data/prepared_columns_processed_v200.csv')

In [19]:
prepared_column_definition_training_data = prepared_columns[['table_id','definition', 'description', 'output', 'display_groups']]

In [20]:
def remove_quotes(s):
    if s.startswith('"') and s.endswith('"'):
        return s[1:-1]  # Remove the first and last character (the double quotes)
    return s  # Return the string unchanged if it doesn't start and end with a double quote

# Use the .loc accessor along with the apply method to modify the 'output' column
prepared_column_definition_training_data.loc[:, 'output'] = prepared_column_definition_training_data['output'].apply(lambda x: remove_quotes(x))

In [21]:
# prepared_column_definition_training_data.to_csv('./data/training/training_data_edw2_column_definitions.csv', index=False)

In [22]:
column_definition_training_prompt = "Given a description of a column in @AvantQueryEDW2, provide the corresponding JSON definition for that column."

In [23]:
def create_col_definition_dict(row):
    return {
        "instruction": column_definition_training_prompt,
        "input": row['output'],
        "output": row['definition']
    }

## Column Definition Training Data Creation

In [24]:

col_definition_data_list = prepared_column_definition_training_data.apply(create_col_definition_dict, axis=1).tolist()

In [25]:
# pd.DataFrame(data_list).to_csv('./data/training/training_data_edw2_column_definitions.csv', index=False)

## Columns Table I.D. (Human Readable Name Definition) Data Creation

In [26]:
column_definition_training_prompt = "Given a description of a column in an @AvantQueryEDW2, provide the corresponding Table I.D. for that column."
def create_table_id_dict(row):
    return {
        "instruction": column_definition_training_prompt,
        "input": row['output'],
        "output": row['table_id']
    }

table_id_data_list = prepared_column_definition_training_data.apply(create_table_id_dict, axis=1)

In [27]:
all_training_data = col_definition_data_list + table_id_data_list.to_list() + report_query_data_list

In [28]:
pd.DataFrame(all_training_data).to_csv('./data/training/training_data_edw2_all.csv', index=False)

In [29]:
pd.DataFrame(all_training_data)

,instruction,input,output
0,Given a description of a column in @AvantQuery...,Avantmetrics: Summarised count of channel orders,"[{""id"": ""fact_order_channel_summary-order_coun..."
1,Given a description of a column in @AvantQuery...,Avantmetrics: Marketing Channel analysis,"[{""id"": ""fact_order_channel_summary-marketing_..."
2,Given a description of a column in @AvantQuery...,Affiliate Classic ID through Avantmetrics,"[{""id"": ""fact_affiliate_type_order_summary-aff..."
3,Given a description of a column in @AvantQuery...,US Holiday Name feature in Performance Display...,"[{""id"": ""dim_us_holidays-holiday_name"", ""name""..."
4,Given a description of a column in @AvantQuery...,Country-specific affiliate networks by AvantLink.,"[{""id"": ""dim_affiliate-affiliate_network"", ""na..."
...,...,...,...
1242,Given the following Avantlink User Query (@Ava...,@AvantQueryEDW2 - Generate a custom report for...,"{""performance_summary"":{""cols"":[{""alias"":""merc..."
1243,Given the following Avantlink User Query (@Ava...,"@AvantQueryEDW2 - Generate a report named ""EPC...","{""performance_summary"":{""cols"":[{""id"":""dim_dat..."
1244,Given the following Avantlink User Query (@Ava...,@AvantQueryEDW2 - Generate a performance summa...,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
1245,Given the following Avantlink User Query (@Ava...,@AvantQueryEDW2 - Generate a Custom Report tha...,"{""performance_summary"":{""cols"":[{""id"":""dim_aff..."


In [30]:
with open('./data/source_data/EDW2_dashboard_objects.json', 'r') as file:
    edw2_dashboard_objects = json.load(file)

In [34]:
trending_widget_objects = edw2_dashboard_objects["trending_widget"]
top_accounts_objects = edw2_dashboard_objects["top_affiliates_widget"]
trending_widget_dict = {}
top_accounts_dict = {}
for category in trending_widget_objects:
    for report in list(trending_widget_objects[category]):  # Using list to avoid RuntimeError
        new_keyname = category + "_" + report
        # This will create/overwrite a key in trending_widget_dict with the new_keyname
        # and set its value to the current report object
        trending_widget_dict[new_keyname] = trending_widget_objects[category].pop(report)

for category in top_accounts_objects:
    for report in list(top_accounts_objects[category]):  # Using list to avoid RuntimeError
        new_keyname = category + "_" + report
        # This will create/overwrite a key in trending_widget_dict with the new_keyname
        # and set its value to the current report object
        top_accounts_dict[new_keyname] = top_accounts_objects[category].pop(report)
        

In [32]:
trending_widget_dict

{'Sales_Sales': {'trending_widget': {'cols': [{'id': 'dim_date-mm_dd_yyyy',
     'name': 'Day',
     'alias': 'mm_dd_yyyy',
     'aggregate': [{'func': 'range'}]},
    {'id': 'calculation',
     'calc': 'sales + adjustments',
     'fact': True,
     'name': 'Sales',
     'vars': {'sales': {'id': 'fact_order_avantlink-order_amount',
       'aggregate': [{'func': 'sum', 'distinct': True}],
       'required_groups': ['sales']},
      'adjustments': {'id': 'fact_order_adjustment-order_combined_adjustment_amount',
       'aggregate': [{'func': 'sum', 'distinct': True}],
       'required_groups': ['sales']}},
     'alias': 'net_sales',
     'format': 'money'}],
   'report_name': 'Custom Report',
   'format': 'json',
   'filters': [{'field': 'dim_date-mm_dd_yyyy',
     'op': 'relative_date',
     'values': [],
     'alias': 'date_filter1',
     'allow_empty': True,
     'to_date': False,
     'count': 30,
     'start': -1,
     'period': 'day'},
    {'field': 'dim_merchant-merchant_uuid',
   